In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)
print(np.__version__)

1.11.0
1.15.2


In [2]:
sess = tf.InteractiveSession()

### We first train a dummy LSTM.

In [3]:
# Make some dummy data.
n_datapoints = 128
n_timesteps = 100
data_dim = 24
labels_dim = 52
x_train = np.random.uniform(size=[n_datapoints, n_timesteps, data_dim])
y_train = np.random.uniform(size=[n_datapoints, n_timesteps, labels_dim])

# Turn y_train to one-hot
y_train = y_train.reshape([-1, labels_dim])
ones_ind = np.argmax(y_train, axis=-1)
y_train = np.zeros(shape=y_train.shape)
y_train[np.arange(n_datapoints*n_timesteps), ones_ind] = 1
y_train = y_train.reshape([n_datapoints, n_timesteps, labels_dim])

print(x_train.shape)
print(y_train.shape)

(128, 100, 24)
(128, 100, 52)


In [4]:
LSTM_LATENT_DIM = 32

inputs = tf.keras.Input(shape=(None, data_dim)) # => (None, None, data_dim)
x = tf.keras.layers.LSTM(units=LSTM_LATENT_DIM, return_sequences=True)(inputs) # => (None, None, LSTM_LATENT_DIM)
outputs = tf.keras.layers.Dense(labels_dim, activation=tf.nn.softmax)(x) # => (None, None, labels_dim)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=1, batch_size=32)

Epoch 1/1
128/128 [==============================] - 1s 5ms/step - loss: 3.9616 - acc: 0.0206


### We can freeze the graph just fine, but importing the frozen graph has an error:

In [5]:
from tensorflow.python.framework import graph_util
frozen_graph_def = graph_util.convert_variables_to_constants(
    sess, sess.graph_def, ['dense/Softmax'])

INFO:tensorflow:Froze 5 variables.
INFO:tensorflow:Converted 5 variables to const ops.


In [6]:
tf.import_graph_def(frozen_graph_def)

ValueError: Input 0 of node import/lstm/while/ReadVariableOp/Enter was passed float from import/lstm/kernel:0 incompatible with expected resource.

### However if we don't convert these variables related to the LSTM, it works fine.

In [7]:
frozen_graph_def_2 = graph_util.convert_variables_to_constants(
    sess, sess.graph_def, ['dense/Softmax'],
    variable_names_blacklist=['lstm/kernel', 'lstm/bias', 'lstm/recurrent_kernel'])

INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.


In [8]:
tf.import_graph_def(frozen_graph_def_2)